# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_reader = pd.read_csv("../output_data/weather_data.csv")
weather_df = csv_reader
weather_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Clouds,Wind
0,Busselton,-33.64,115.35,78.49,23,0,13.78
1,Ushuaia,-54.81,-68.31,48.20,87,90,17.22
2,Thompson,55.74,-97.86,-16.60,75,40,5.82
3,Nishihara,35.74,139.53,40.98,93,90,8.05
4,Punta Arenas,-53.16,-70.91,53.60,66,100,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [262]:
gmaps.configure(api_key=g_key)

In [263]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [264]:
# A max temperature lower than 76 degrees but higher than 50.
# Wind speed less than 10 mph.
# Zero cloudiness.
length = len(weather_df)
x = 0

while (x-1) < length:
    if weather_df['Temperature'][x] > 76 :
        weather_df = weather_df.drop([x])
        length = length - 1
        x = x + 1
    elif weather_df['Temperature'][x] < 50:
        weather_df = weather_df.drop([x])
        length = length - 1
        x = x + 1
    elif weather_df['Clouds'][x] > 0:
        weather_df = weather_df.drop([x])
        length = length - 1
        x = x + 1
    elif weather_df['Wind'][x] > 10:
        weather_df = weather_df.drop([x])
        length = length - 1
        x = x + 1        
    else:
        x = x + 1

In [265]:
weather_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Clouds,Wind
6,Hermanus,-34.42,19.24,64.11,96,0,3.38
31,Bredasdorp,-34.53,20.04,65.16,94,0,3.36
37,Chuy,-33.69,-53.46,64.92,62,0,8.43
39,Kutum,14.20,24.66,54.57,29,0,2.82
47,Salalah,17.01,54.10,75.20,53,0,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [270]:
hotel_df = pd.DataFrame()
hotel_df = weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Clouds,Wind,Hotel Name
6,Hermanus,-34.42,19.24,64.11,96,0,3.38,
31,Bredasdorp,-34.53,20.04,65.16,94,0,3.36,
37,Chuy,-33.69,-53.46,64.92,62,0,8.43,
39,Kutum,14.20,24.66,54.57,29,0,2.82,
47,Salalah,17.01,54.10,75.20,53,0,9.17,


In [274]:
# set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [277]:
hotel_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Clouds,Wind,Hotel Name
6,Hermanus,-34.42,19.24,64.11,96,0,3.38,Birkenhead House
31,Bredasdorp,-34.53,20.04,65.16,94,0,3.36,Bredasdorp Country Manor
37,Chuy,-33.69,-53.46,64.92,62,0,8.43,Rivero Hotel Chuí
39,Kutum,14.20,24.66,54.57,29,0,2.82,
47,Salalah,17.01,54.10,75.20,53,0,9.17,Al Baleed Resort Salalah by Anantara


In [286]:
hotel_df.rename(columns={"Unnamed: 0": "City"})

,City,Latitude,Longitude,Temperature,Humidity,Clouds,Wind,Hotel Name
6,Hermanus,-34.42,19.24,64.11,96,0,3.38,Birkenhead House
31,Bredasdorp,-34.53,20.04,65.16,94,0,3.36,Bredasdorp Country Manor
37,Chuy,-33.69,-53.46,64.92,62,0,8.43,Rivero Hotel Chuí
39,Kutum,14.20,24.66,54.57,29,0,2.82,
47,Salalah,17.01,54.10,75.20,53,0,9.17,Al Baleed Resort Salalah by Anantara
97,San Pedro,-33.68,-59.66,69.33,61,0,1.99,Barrancas Hotel
138,San Patricio,-26.98,-56.83,71.42,31,0,4.00,
152,Cidreira,-30.17,-50.22,74.59,71,0,9.28,Hotel Castelo
187,Coquimbo,-29.95,-71.34,62.60,82,0,5.82,Enjoy Coquimbo Hotel de la Bahía
200,Mapastepec,15.44,-92.90,73.94,60,0,5.10,Hotel Diana


In [287]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [281]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

# Display Map

NameError: name 'hotel_info' is not defined